In [1]:
import sys; sys.path.append('..')
import prosodic2 as p

In [2]:
txt="""
If I were a dead leaf thou mightest bear;
If I were a swift cloud to fly with thee;
A wave to pant beneath thy power, and share

The impulse of thy strength, only less free
Than thou, O uncontrollable! If even
I were as in my boyhood, and could be

The comrade of thy wanderings over Heaven,
As then, when to outstrip thy skiey speed
Scarce seem'd a vision; I would ne'er have striven

As thus with thee in prayer in my sore need.
Oh, lift me as a wave, a leaf, a cloud!
I fall upon the thorns of life! I bleed!

A heavy weight of hours has chain'd and bow'd
One too like thee: tameless, and swift, and proud.
"""

In [3]:
bigtxt = '\n\n'.join([txt for n in range(100)])
len([x for x in bigtxt.split('\n') if x.strip()])

1400

In [4]:
df_phon=p.parse_phon(txt,incl_alt=True,num_proc=4)
df_phon

Tokenizing and syllabifying [x4]: 100%|██████████| 14/14 [00:00<00:00, 18.14it/s]


,stanza_i,line_i,word_i,word_ipa_i,syll_i,word_str,word_ipa,syll_ipa,syll_str
0,0,0,0,0,0,If,'ɪf,'ɪf,If
1,0,0,0,1,0,If,ɪf,ɪf,If
2,0,0,1,0,0,I,'aɪ,'aɪ,I
3,0,0,2,0,0,were,wɛː,wɛː,were
4,0,0,2,1,0,were,'wɛː,'wɛː,were
...,...,...,...,...,...,...,...,...,...
9,4,1,7,1,0,and,ənd,ənd,and
10,4,1,8,0,0,swift,'swɪft,'swɪft,swift
11,4,1,10,0,0,and,'ænd,'ænd,and
12,4,1,10,1,0,and,ənd,ənd,and


In [20]:

def product(*args):
	if not args:
		return iter(((),)) # yield tuple()
	return (items + (item,)
		for items in product(*args[:-1]) for item in args[-1])

def line2combos(line_df):
    word_rows = [
        set(list(zip(word_df.word_i, word_df.word_ipa_i, word_df.word_ipa)))                
        # set(list(zip(word_df.word_i, word_df.word_ipa_i, word_df.word_ipa)))                
        for word_i,word_df in sorted(line_df.groupby('word_i'))
    ]

    # divide at blank points
    word_rows_div=[]
    word_row_div = []
    for wordforms in word_rows:
        # split into phrases?
        if len(wordforms)==1 and not list(wordforms)[0][-1].strip():
            if len(word_row_div)>=MIN_WORDS_IN_PHRASE:
                word_rows_div.append(word_row_div)
                word_row_div=[]
        else:
            word_row_div.append(wordforms)
    if word_row_div: word_rows_div.append(word_row_div)

    from pprint import pprint
    
    # gen combos
    all_combos = []
    for word_row_div in word_rows_div:
        combos = list(product(*word_row_div))
        all_combos.extend(combos)
    
    # loop
    combo_dfs=[]
    ldf=line_df.set_index(['word_i','word_ipa_i'])
    for combo in all_combos:
        # print(combo)
        combo_df=ldf.loc[[(cx[0],cx[1]) for cx in combo]].reset_index()
        # display(combo_df)
        combo_dfs.append(combo_df)
        # break
    return combo_dfs

In [21]:
first_line = df_phon.query('stanza_i==0 & line_i==0')
# first_line

In [25]:
combo_dfs=line2combos(first_line)
len(combo_dfs)

8

In [26]:
def all_metrical_combos(df_combo):
    wordsylls=sorted(list(set(zip(df_combo.word_i,df_combo.syll_i))))

    wordsyllsmeter = [
        [tuple(list(wsyll)+['w']), tuple(list(wsyll)+['s'])]
        for wsyll in wordsylls
    ]

    wordsyllsmeter_combos = list(product(*wordsyllsmeter))
    
    # disallow stretches of 3
    wordsyllsmeter_combos = [
        combo
        for combo in wordsyllsmeter_combos
        if ''.join([x[-1] for x in combo]).count('www')==0
        and ''.join([x[-1] for x in combo]).count('sss')==0
    ]
    return wordsyllsmeter_combos

In [28]:
# all_metrical_combos(combo_dfs[0])

In [ ]:
def parse_phonological_combo(df_combo):
    wordsyllsmeter_combos = self.all_metrical_combos(df_combo)
    dfwsc = df_combo.set_index(['word_i','syll_i'])
    dfall=pd.DataFrame()
    # for wi,wsc in enumerate(tqdm(wordsyllsmeter_combos,desc='Parsing line combos',position=1)):
    for wi,wsc in enumerate(wordsyllsmeter_combos):#,desc='Parsing line combos',position=1)):
        ws2mtr=dict(([((w,s),c) for w,s,c in wsc]))
        df_mcombo = pd.DataFrame(dfwsc.loc[ws2mtr.keys()].reset_index().sort_values(['word_i','syll_i']))
        df_mcombo['parse']=[ws2mtr.get((w,s)) for w,s in zip(df_mcombo.word_i,df_combo.syll_i)]
        
        df_mcombo_parsed = self.parse_metrical_combo(df_mcombo)
        df_mcombo_parsed['meter_combo_i']=int(wi)
        df_mcombo_parsed['meter_combo_str']=''.join(df_mcombo['parse'])
        df_mcombo_parsed['meter_combo_str_w']='|'.join(
            ''.join(grp.sort_values('syll_i').parse)
            for grpname,grp in df_mcombo.groupby('word_i')
        )
        # df_mcombo_parsed['stress_parse']=self.combo2stress(df_mcombo.word_ipa)
        # df_mcombo_parsed['line_combo_ipa']=' '.join(df_mcombo_parsed.word_ipa)
        cstrcols=[col for col in df_mcombo_parsed.columns if col.startswith('*')]
        df_mcombo_parsed['*total']=df_mcombo_parsed[cstrcols].sum(axis=1)
        dfcols=['stanza_i','line_i','word_i','word_ipa_i','syll_i','stress_parse','line_combo_ipa','meter_combo_i','meter_combo_str','meter_combo_str_w','parse']
        dfall=dfall.append(df_mcombo_parsed[
            dfcols+cstrcols+['*total']
        ])
    return dfall